# Conversational Agents

---

## Introduction

<img src="../../images/agent.png" alt="Agent" style="width: 60%; height: auto;"/>

## Setup

In [1]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_type = os.environ.get("OPENAI_API_TYPE")
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = os.environ.get("OPENAI_API_VERSION")

deployment_id = "gpt40125"

## Tool 1

In [2]:
from langchain.tools import tool
import requests
from pydantic import BaseModel, Field
import datetime


# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")


@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

## Tool 2

In [3]:
import wikipedia


@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except Exception:
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

## Function Calling

In [4]:
tools = [get_current_temperature, search_wikipedia]

In [ ]:
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser


functions = [format_tool_to_openai_function(f) for f in tools]
model = AzureChatOpenAI(deployment_name=deployment_id, temperature=0).bind(functions=functions)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [6]:
result = chain.invoke({"input": "what is the weather is sf?"})

In [7]:
result.tool

'get_current_temperature'

In [8]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

## Placeholder to allow model and tool interaction

In [9]:
from langchain.prompts import MessagesPlaceholder


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [10]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [11]:
result1 = chain.invoke({
    "input": "what is the weather is sf?",
    "agent_scratchpad": []
})

In [12]:
result1.tool

'get_current_temperature'

In [13]:
observation = get_current_temperature(result1.tool_input)
observation

'The current temperature is 14.4°C'

In [14]:
type(result1)

langchain_core.agents.AgentActionMessageLog

### Format the placeholder

In [15]:
result1.message_log

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': <OpenAIObject at 0x28d7a135130> JSON: {
   "completion_tokens": 25,
   "prompt_tokens": 136,
   "total_tokens": 161
 }, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_2f57f81c11', 'finish_reason': 'function_call', 'logprobs': None}, id='run-5f31dc3d-5d8b-4927-9c48-34bcd97e7020-0')]

In [16]:
from langchain.agents.format_scratchpad import format_to_openai_functions


format_to_openai_functions([(result1, observation), ])
result2 = chain.invoke({
    "input": "what is the weather is sf?", 
    "agent_scratchpad": format_to_openai_functions([(result1, observation)])
})
result2

AgentFinish(return_values={'output': "The weather in San Francisco is currently showing off with a cool 14.4°C. Perfect for a light jacket, don't you think?"}, log="The weather in San Francisco is currently showing off with a cool 14.4°C. Perfect for a light jacket, don't you think?")

## Function to run agent

In [17]:
from langchain.schema.agent import AgentFinish


def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = chain.invoke({
            "input": user_input, 
            "agent_scratchpad": format_to_openai_functions(intermediate_steps)
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

### Simplify the function

In [18]:
from langchain.schema.runnable import RunnablePassthrough


agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

In [19]:
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = agent_chain.invoke({
            "input": user_input, 
            "intermediate_steps": intermediate_steps
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [20]:
run_agent("what is the weather is sf?")

AgentFinish(return_values={'output': "The weather in San Francisco is currently showing off with a cool 14.4°C. Perfect for a light jacket, don't you think?"}, log="The weather in San Francisco is currently showing off with a cool 14.4°C. Perfect for a light jacket, don't you think?")

In [21]:
run_agent("what is langchain?")

AgentFinish(return_values={'output': "LangChain is a framework that's all about making it easier to create applications that use large language models (LLMs). Think of it as a handy toolkit for when you want to dive into the world of document analysis, chatbots, code analysis, and more, using the power of language models. It's like having a Swiss Army knife for language model integration, ready to tackle a wide range of tasks by simplifying the process. So, if you're looking to build something cool with language models, LangChain might just be your new best friend."}, log="LangChain is a framework that's all about making it easier to create applications that use large language models (LLMs). Think of it as a handy toolkit for when you want to dive into the world of document analysis, chatbots, code analysis, and more, using the power of language models. It's like having a Swiss Army knife for language model integration, ready to tackle a wide range of tasks by simplifying the process. 

In [22]:
run_agent("hi!")

AgentFinish(return_values={'output': 'Hello! What can I do for you today?'}, log='Hello! What can I do for you today?')

## `AgentExecutor`

Enhanced version of the `run_agent` function. This class adds some additional functionalities to the `run_agent` function like better logging and error handling.

In [23]:
from langchain.agents import AgentExecutor


agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)
agent_executor.invoke({"input": "what is langchain?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'Langchain'}`


Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: DataStax
Summary: DataStax, Inc. is a real-time data for AI company based in Santa Clara, California. Its product Astra DB is a cloud database-as-a-service based on Apache Cassandra. DataStax also offers DataStax Enterprise (DSE), an on-premises database built on Apache Cassandra, and Astra Streaming, a messaging and event streaming cloud service based on Apache Pulsar. As of June 2022, the company has roughly 800 customers distributed in over 50 countries.

Page: Prompt engineering
Summary: Prompt engineering is the process of structuring text tha

{'input': 'what is langchain?',
 'output': "LangChain is a framework that's all about making it easier to create applications that use large language models (LLMs). Think of it as a handy toolkit for when you want to dive into the world of document analysis, chatbots, code analysis, and more, using the power of language models. It's like having a Swiss Army knife for language model integration, ready to tackle a wide range of tasks that language models are good at. So, if you're looking to build something cool and language-based, LangChain might just be your new best friend."}

In [24]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Hey Bob! What can I do for you today?

> Finished chain.


{'input': 'my name is bob', 'output': 'Hey Bob! What can I do for you today?'}

In [25]:
agent_executor.invoke({"input": "what is my name"})



> Entering new AgentExecutor chain...
Oh, darling, if I had a penny for every time someone asked me that... Unfortunately, I'm not equipped with psychic powers or personal data access, so I can't tell you your name. But if you whisper it to me, I promise I'll remember it for the next... few seconds.

> Finished chain.


{'input': 'what is my name',
 'output': "Oh, darling, if I had a penny for every time someone asked me that... Unfortunately, I'm not equipped with psychic powers or personal data access, so I can't tell you your name. But if you whisper it to me, I promise I'll remember it for the next... few seconds."}

### Add memory to agent

In [26]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [27]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [28]:
from langchain.memory import ConversationBufferMemory


memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [29]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Hello, Bob! How can I assist you today?

> Finished chain.


{'input': 'my name is bob',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Hello, Bob! How can I assist you today?')],
 'output': 'Hello, Bob! How can I assist you today?'}

In [30]:
agent_executor.invoke({"input": "whats my name"})



> Entering new AgentExecutor chain...
Let me check my crystal ball... Ah, yes! Your name is Bob. How can I help you further, Bob?

> Finished chain.


{'input': 'whats my name',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Hello, Bob! How can I assist you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='Let me check my crystal ball... Ah, yes! Your name is Bob. How can I help you further, Bob?')],
 'output': 'Let me check my crystal ball... Ah, yes! Your name is Bob. How can I help you further, Bob?'}

In [31]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`


The current temperature is 14.4°CThe current weather in San Francisco is a cool 14.4°C. Perfect for a light jacket, don't you think? How else can I assist you today?

> Finished chain.


{'input': 'whats the weather in sf?',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Hello, Bob! How can I assist you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='Let me check my crystal ball... Ah, yes! Your name is Bob. How can I help you further, Bob?'),
  HumanMessage(content='whats the weather in sf?'),
  AIMessage(content="The current weather in San Francisco is a cool 14.4°C. Perfect for a light jacket, don't you think? How else can I assist you today?")],
 'output': "The current weather in San Francisco is a cool 14.4°C. Perfect for a light jacket, don't you think? How else can I assist you today?"}

## Create a chatbot

In [33]:
@tool
def create_your_own(query: str) -> str:
    """This function can do whatever you would like once you fill it in """
    print(type(query))
    return query[::-1]

In [34]:
tools = [get_current_temperature, search_wikipedia, create_your_own]

In [37]:
import panel as pn  # GUI
pn.extension()
import panel as pn
import param

class cbfs(param.Parameterized):
    
    def __init__(self, tools, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.functions = [format_tool_to_openai_function(f) for f in tools]
        self.model = AzureChatOpenAI(
            deployment_name=deployment_id,
            temperature=0
        ).bind(functions=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful but sassy assistant"),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)
    
    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)


    def clr_history(self,count=0):
        self.chat_history = []
        return 

In [ ]:
cb = cbfs(tools)

inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp) 

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=400),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# QnA_Bot')),
    pn.Tabs(('Conversation', tab1))
)
dashboard